<a href="https://colab.research.google.com/github/Davron030901/Data_Science/blob/main/Bankga_mijozlarining_qarzni_qaytarish_prognozi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
file_id = '1hvcnbiiNUBWkmgMQaGOEY33mhrloJ5xZ'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={file_id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={file_id}" -O file.zip
!rm -rf /tmp/cookies.txt

# Unzip the file
!unzip -o file.zip

--2025-03-13 02:10:42--  https://docs.google.com/uc?export=download&confirm=&id=1hvcnbiiNUBWkmgMQaGOEY33mhrloJ5xZ
Resolving docs.google.com (docs.google.com)... 74.125.202.102, 74.125.202.100, 74.125.202.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.202.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1hvcnbiiNUBWkmgMQaGOEY33mhrloJ5xZ&export=download [following]
--2025-03-13 02:10:42--  https://drive.usercontent.google.com/download?id=1hvcnbiiNUBWkmgMQaGOEY33mhrloJ5xZ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.201.132, 2607:f8b0:4001:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.201.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 651320 (636K) [application/octet-stream]
Saving to: ‘file.zip’

file.zip            100%[===================>] 636.05K  --.-KB/s   

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train['BalanceSalaryRatio'] = df_train['Balance'] / (df_train['EstimatedSalary'] + 1)
df_test['BalanceSalaryRatio'] = df_test['Balance'] / (df_test['EstimatedSalary'] + 1)

In [ ]:
numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
                     'HasCrCard', 'IsActiveMember', 'EstimatedSalary','BalanceSalaryRatio']
categorical_features = ['Geography', 'Gender']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

In [ ]:
X_train_full = df_train.drop(['Exited', 'CustomerId', 'Surname', 'id'], axis=1)
y_train_full = df_train['Exited']
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

In [ ]:
param_grid = {
    'classifier__n_estimators': [80, 100],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [4, 5],
    'classifier__subsample': [0.8, 1.0],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

In [ ]:
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train_split, y_train_split)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['CreditScore',
                                                                          'Age',
                                                                          'Tenure',
                                                                          'Balance',
                                                                          'NumOfProducts',
                                                                          'HasCrCard',
                                                                          'IsActiveMember',
                                                                          'EstimatedSalary',
                                                                          'BalanceSalaryRatio']),
                                                                        ('cat',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Geography',
                                                                          'Gender'])])),
                                       ('classifier',
                                        GradientBoostingClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__learning_rate': [0.01, 0.1],
                         'classifier__max_depth': [4, 5],
                         'classifier__min_samples_leaf': [1, 2],
                         'classifier__min_samples_split': [2, 5],
                         'classifier__n_estimators': [80, 100],
                         'classifier__subsample': [0.8, 1.0]},
             scoring='roc_auc', verbose=1)

In [ ]:
print("Best parameters:", grid_search.best_params_)
print("Best AUC-ROC:", grid_search.best_score_)

Best parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 4, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 80, 'classifier__subsample': 0.8}
Best AUC-ROC: 0.9285820798387291


In [ ]:
best_model = grid_search.best_estimator_
y_val_pred_proba = best_model.predict_proba(X_val_split)[:, 1]
print(f"Validation AUC-ROC: {roc_auc_score(y_val_split, y_val_pred_proba):.4f}")

Validation AUC-ROC: 0.9397


In [ ]:
X_test = df_test.drop(['id', 'CustomerId', 'Surname'], axis=1)
test_predictions = best_model.predict_proba(X_test)[:, 1]

In [ ]:
submission = pd.DataFrame({'id': df_test['id'], 'Exited': np.round(test_predictions, 1)})
submission.to_csv('submission.csv', index=False)

In [ ]:
df=pd.read_csv('submission.csv')
df.head(10)

,id,Exited
0,15000,0.1
1,15001,0.2
2,15002,0.0
3,15003,0.1
4,15004,0.0
5,15005,0.4
6,15006,0.1
7,15007,0.7
8,15008,0.0
9,15009,0.5


In [ ]:
df.tail(10)

,id,Exited
9990,24990,0.0
9991,24991,0.0
9992,24992,0.1
9993,24993,0.9
9994,24994,0.0
9995,24995,0.1
9996,24996,0.1
9997,24997,1.0
9998,24998,0.0
9999,24999,0.1


In [ ]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(learning_rate=0.1, max_depth=4, min_samples_split=2, n_estimators=80, subsample=0.8,random_state=42))  # Model
])

In [ ]:
model.fit(X_train_split, y_train_split)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary',
                                                   'BalanceSalaryRatio']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['Geography', 'Gender'])])),
                ('classifier',
                 GradientBoostingClassifier(max_depth=4, n_estimators=80,
                                            random_state=42, subsample=0.8))])

In [ ]:
y_val_pred_proba = model.predict_proba(X_val_split)[:, 1]
print(f"Validation AUC-ROC: {roc_auc_score(y_val_split, y_val_pred_proba):.4f}")

Validation AUC-ROC: 0.9404


In [ ]:
X_test = df_test.drop(['id', 'CustomerId', 'Surname'], axis=1)
test_predictions = model.predict_proba(X_test)[:, 1]

In [ ]:
submission = pd.DataFrame({'id': df_test['id'], 'Exited': test_predictions})
submission.to_csv('submission.csv', index=False)

In [ ]:
df=pd.read_csv('submission.csv')
df.head(10)

,id,Exited
0,15000,0.072339
1,15001,0.126207
2,15002,0.036217
3,15003,0.125669
4,15004,0.015955
5,15005,0.378513
6,15006,0.069986
7,15007,0.633904
8,15008,0.012880
9,15009,0.502686
